In [1]:
numCustomersTypes = 25
numObjectives = 11

import pandas as pd
import numpy as np

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)

wts = [np.random.random(numObjectives) for _ in range(numCustomersTypes)]


# Create the DataFrame
df = pd.DataFrame({
    'Name': names,
    'Quantity': qty,
    'Weights': wts
})
df.to_csv('../resources/customers.csv', index=False)
df

,Name,Quantity,Weights
0,CustomerType1,788,"[0.7351336666549761, 0.4845961236417954, 0.932..."
1,CustomerType2,923,"[0.790956389442883, 0.6145504615266982, 0.2137..."
2,CustomerType3,348,"[0.6653154489288534, 0.176104331325306, 0.3504..."
3,CustomerType4,295,"[0.9664575280860574, 0.9148165287798593, 0.418..."
4,CustomerType5,586,"[0.060486775842366836, 0.8441330421511948, 0.8..."
5,CustomerType6,521,"[0.6111863251614801, 0.7557145959932678, 0.897..."
6,CustomerType7,142,"[0.900232754457577, 0.01412840697372253, 0.653..."
7,CustomerType8,313,"[0.6657938173209008, 0.8146547395827547, 0.403..."
8,CustomerType9,946,"[0.07086346336512117, 0.006791767833699502, 0...."
9,CustomerType10,240,"[0.6465972200417225, 0.07024750470637897, 0.28..."
